**Домашнее задание 1. Тональность отзывов.**

Будем анализировать отзывы на книги с "Лабиринта"!

Краулером пройдёмся по сайту "Лабиринта" и будем парсить рецензии с помощью BeautifulSoup. Сначала распарсим одну страницу.

In [2]:
! pip install requests
! pip install bs4
! pip install fake_useragent
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

In [3]:
import re
import numpy as np

In [4]:
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}
print(headers)

session = requests.session()
response = session.get('https://www.labirint.ru', headers=headers)

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "C:\Users\nasty\anaconda3\lib\site-packages\fake_useragent\utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "C:\Users\nasty\anaconda3\lib\site-packages\fake_useragent\utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


{'User-Agent': 'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36'}


In [5]:
url = f'https://www.labirint.ru/reviews/show/2412313/'
req = session.get(url, headers=headers)
page = req.content

In [6]:
soup = BeautifulSoup(page, 'html.parser')

print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html prefix="ya: http://webmaster.yandex.ru/vocabularies/" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<script>
if(typeof console === 'undefined') {
	var console = {
		time: function (a,b) {},
		timeLog: function (a,b) {}
	};
}
if(!('time' in console)) {
	console.time = function (a,b) {};
}
if(!('timeLog' in console)) {
	console.timeLog = function (a,b) {};
}
console.time('cart', 'start');</script>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://css.labirint.ru" rel="dns-prefetch"/><link href="https://js.labirint.ru" rel="dns-prefetch"/><link href="https://img.labirint.ru" rel="dns-prefetch"/><link href="https://img1.labirint.ru" rel="dns-prefetch"/><link href="https://img2.labirint.ru" rel="d

**Создание таблицы с данными**

In [7]:
data = pd.DataFrame()
data

""


**Парсинг оценок и отзывов (на примере одной страницы)**

In [8]:
review = soup.find_all('div', class_='comment-text content-comments')

In [9]:
review

[<div class="comment-text content-comments">
 <!-- Текст -->
 <div>
 <p>Романтическая новелла от эксперта по отношениям Гэри Чепмена. <br/>
 <br/>
 Марли и Джейкоб поженились в канун Рождества. Но спустя 20 лет, решили развестись. <br/>
 <br/>
 Они едут к юристу, чтобы закончить свою историю и подписать необходимые документы о разводе. Попадают в снежную бурю и Джейкоб не справляется с управлением, их заносит, они уходят в кювет. <br/>
 <br/>
 Марли приходит в себя, не видит рядом мужа, отправляется на его поиски. Видит дом, ей открывает пожилой мужчина и тут начинается что-то вообще непонятное мне…<br/>
 <br/>
 Книга меня расстроила полностью. Такое красивое оформление, приятная на ощупь обложка, качественная бумага и отличный шрифт. Но содержание - это вынос мозга, вообще не понятно, что я читаю. Радует, что прочитать книгу можно за 3-4 часа и забыть про нее. <br/>
 <br/>
 Общий балл от меня - 3/5 (и это только за оформление книги).</p>
 </div>
 <!-- Картинки -->
 <!-- Конец картинки

In [10]:
review_clean_reg = re.compile('<p>((?:.|\n)*)<\/p>')

In [11]:
review_clean = re.findall(review_clean_reg, str(review))
review_clean

['Романтическая новелла от эксперта по отношениям Гэри Чепмена. <br/>\n<br/>\nМарли и Джейкоб поженились в канун Рождества. Но спустя 20 лет, решили развестись. <br/>\n<br/>\nОни едут к юристу, чтобы закончить свою историю и подписать необходимые документы о разводе. Попадают в снежную бурю и Джейкоб не справляется с управлением, их заносит, они уходят в кювет. <br/>\n<br/>\nМарли приходит в себя, не видит рядом мужа, отправляется на его поиски. Видит дом, ей открывает пожилой мужчина и тут начинается что-то вообще непонятное мне…<br/>\n<br/>\nКнига меня расстроила полностью. Такое красивое оформление, приятная на ощупь обложка, качественная бумага и отличный шрифт. Но содержание - это вынос мозга, вообще не понятно, что я читаю. Радует, что прочитать книгу можно за 3-4 часа и забыть про нее. <br/>\n<br/>\nОбщий балл от меня - 3/5 (и это только за оформление книги).']

In [12]:
data['review'] = review_clean

In [13]:
data

,review
0,Романтическая новелла от эксперта по отношения...


In [14]:
reg_note = re.compile(r'Оценка товара:(?:.|\n)*')

In [15]:
note = re.findall(reg_note, str(soup))
note[0]

'Оценка товара:\r\n\t\t\t\t\t</div>\n<div class="form-inp" title="Плохо">\n<span class="stars" id="mark-stars"><span class="star full header-sprite" id="mark-view-star1"></span><span class="star full header-sprite" id="mark-view-star2"></span><span class="star full header-sprite" id="mark-view-star3"></span><span class="star empty header-sprite" id="mark-view-star4"></span><span class="star empty header-sprite" id="mark-view-star5"></span><span class="star empty header-sprite" id="mark-view-star6"></span><span class="star empty header-sprite" id="mark-view-star7"></span><span class="star empty header-sprite" id="mark-view-star8"></span><span class="star empty header-sprite" id="mark-view-star9"></span><span class="star empty header-sprite" id="mark-view-star10"></span>\n</span>\r\n\t\t\t\t\t\t\t\t\xa0\r\n                        3\t\t\t\t\t</div>\n</div>\n<div class="form-cont">\n<div class="form-txt">\r\n\t\t\t\t\t\tКачество печати:\r\n\t\t\t\t\t</div>\n<div class="form-inp" title="Луч

In [16]:
note = re.search(r"title=\"(.*)\">", note[0]).group(1)
note

'Плохо'

In [17]:
data['note'] = note

In [18]:
data

,review,note
0,Романтическая новелла от эксперта по отношения...,Плохо


**Краулер и парсинг всех страниц**

Работает довольно долго, поэтому можно прервать выполнение этой ячейки раньше (когда наберётся по 60+10=70 отзывов каждого класса)! (Просто отрицательные отзывы очень медленно набираются, их в разы меньше, чем положительных)

In [73]:
for i in range(2412314, 24131300):
    url = 'https://www.labirint.ru/reviews/show/'+str(i)+'/'
    req = session.get(url, headers=headers)
    page = req.content
    soup = BeautifulSoup(page, 'html.parser')
    review = soup.find_all('div', class_='comment-text content-comments')
    note = re.findall(reg_note, str(soup))
    if review and note:
        review_clean = re.findall(review_clean_reg, str(review))[0]
        note = re.search(r"title=\"(.*)\">", note[0]).group(1)
        row = [review_clean, note]
        data.loc[len(data)] = row

KeyboardInterrupt: 

Условие на существование отзыва и оценки нужно потому, что на Лабиринте они не не являются обязательными для публикации одновременно. Тольок отзыв или только оценка не могут стать частью нашей выборки.

In [74]:
data

,review,note
0,Романтическая новелла от эксперта по отношения...,negative
1,Сильно и очень глубоко! Книга о необходимости ...,positive
2,Честно говоря разочарована. Данную книгу скоре...,negative
4,Очередной весьма приличный триллер про доблест...,positive
5,Я решила обновить свое старое издание любимой ...,positive
...,...,...
218,"Вот наконец вышел «третий том», третья заверша...",positive
219,"Интересный триллер, балансирующий между истори...",positive
220,"Книга очень понравилась. Рекомендую всем тем, ...",positive
221,Потрясающий труд аналогов которому я не видел....,positive


In [75]:
data['note'].unique()

array(['negative', 'positive', 'Отлично'], dtype=object)

In [76]:
data = data.replace({'Хорошо': 'positive', 'Лучше не бывает!':'positive', 'Отлично':'positive', 'Плохо':'negative', 'Очень плохо':'negative', 'Хуже не бывает!':'negative'})

In [77]:
data = data[(data['note'] == 'positive') | (data['note'] == 'negative')]

In [78]:
data

,review,note
0,Романтическая новелла от эксперта по отношения...,negative
1,Сильно и очень глубоко! Книга о необходимости ...,positive
2,Честно говоря разочарована. Данную книгу скоре...,negative
4,Очередной весьма приличный триллер про доблест...,positive
5,Я решила обновить свое старое издание любимой ...,positive
...,...,...
218,"Вот наконец вышел «третий том», третья заверша...",positive
219,"Интересный триллер, балансирующий между истори...",positive
220,"Книга очень понравилась. Рекомендую всем тем, ...",positive
221,Потрясающий труд аналогов которому я не видел....,positive


In [79]:
data['note'].value_counts()

positive    177
negative     20
Name: note, dtype: int64

In [80]:
pos = data[data['note'] == 'positive'][:60]
pos

,review,note
1,Сильно и очень глубоко! Книга о необходимости ...,positive
4,Очередной весьма приличный триллер про доблест...,positive
5,Я решила обновить свое старое издание любимой ...,positive
6,"Просто цитата. ""Передовые газет были ужасны – ...",positive
8,Роман «<strong>Джейн Эйр</strong>» был прочита...,positive
9,Хороша книга. Глубокая расшифровка карт. Краси...,positive
10,Единственная книга на рынке с адекватными реце...,positive
11,Просто потрясающая книга. Информация изложена ...,positive
12,Наткнулась на данную книгу совершенно случайно...,positive
13,"Книга небольшого формата,бумага-белый, плотный...",positive


In [82]:
neg = data[data['note'] == 'negative'][:10]
neg

,review,note
0,Романтическая новелла от эксперта по отношения...,negative
2,Честно говоря разочарована. Данную книгу скоре...,negative
7,"Книга показалась очень скучной, сюжет ни о чем...",negative
48,"После прочтения бесплатного отрывка, еще раз в...",negative
57,Я уже боюсь брать книги современных российских...,negative
87,книга шизофреника русофоба. это не научный тру...,negative
88,Книга слабая. И много опечаток.,negative
89,"Неочень книга, и по информативности и по качес...",negative
90,"Неплохой сборник советов по ЗОЖ, читается легк...",negative
104,"Очень пожалела, что купила эти рассказы детям....",negative


In [83]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nasty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [84]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [85]:
pos_bag = {}
for row in pos['review']:
    row = re.sub('<.*>', '', row)
    tokens = nltk.word_tokenize(row)
    for token in tokens:
        if token.isalpha():
            token = token.lower()
            parsed = morph.parse(token)[0]
            if parsed.normal_form in pos_bag:
                pos_bag[parsed.normal_form] += 1
            else:
                pos_bag[parsed.normal_form] = 1
            
print(pos_bag)

{'сильно': 1, 'и': 106, 'очень': 35, 'глубоко': 1, 'книга': 72, 'о': 22, 'необходимость': 1, 'личный': 1, 'свобода': 1, 'подросток': 1, 'токсичный': 1, 'родитель': 3, 'чудесный': 3, 'разный': 3, 'одноклассник': 1, 'педагог': 1, 'от': 8, 'бог': 1, 'нет': 6, 'читать': 11, 'обязательно': 1, 'в': 73, 'первый': 5, 'очередь': 1, 'очередной': 1, 'весьма': 1, 'приличный': 1, 'триллер': 1, 'про': 1, 'доблестный': 1, 'ричер': 1, 'который': 8, 'путешествовать': 1, 'по': 16, 'америка': 1, 'то': 6, 'дело': 3, 'влипать': 1, 'криминальный': 1, 'история': 7, 'но': 24, 'всегда': 2, 'побеждать': 1, 'вот': 3, 'тут': 1, 'не': 42, 'мочь': 5, 'завершиться': 1, 'поражение': 1, 'злодей': 1, 'среди': 1, 'кстати': 3, 'фигурировать': 1, 'злодейский': 1, 'украинец': 1, 'я': 39, 'решить': 4, 'обновить': 1, 'свой': 5, 'старое': 1, 'издание': 9, 'любимый': 3, 'хотеть': 3, 'приобрести': 3, 'перевод': 4, 'оформление': 3, 'иллюстрация': 14, 'это': 30, 'так': 12, 'покорить': 1, 'что': 32, 'именно': 4, 'он': 17, 'пусть':

In [86]:
neg_bag = {}
for row in neg['review']:
    row = re.sub('<.*>', '', row)
    tokens = nltk.word_tokenize(row)
    for token in tokens:
        if token.isalpha():
            token = token.lower()
            parsed = morph.parse(token)[0]
            if parsed.normal_form in neg_bag:
                neg_bag[parsed.normal_form] += 1
            else:
                neg_bag[parsed.normal_form] = 1
            
print(neg_bag)

{'романтический': 1, 'новелла': 1, 'от': 3, 'эксперт': 1, 'по': 7, 'отношение': 1, 'гэри': 1, 'чепмена': 1, 'марля': 2, 'и': 28, 'джейкоб': 2, 'пожениться': 1, 'в': 20, 'канун': 1, 'рождество': 1, 'но': 5, 'спустя': 1, 'год': 1, 'решить': 1, 'развестись': 1, 'они': 6, 'ехать': 1, 'к': 2, 'юрист': 1, 'чтобы': 5, 'закончить': 1, 'свой': 3, 'история': 4, 'подписать': 1, 'необходимый': 1, 'документ': 1, 'о': 3, 'развод': 1, 'попадать': 1, 'снежный': 1, 'буря': 1, 'не': 16, 'справляться': 1, 'с': 5, 'управление': 1, 'заносить': 1, 'уходить': 1, 'кювета': 1, 'приходить': 1, 'себя': 3, 'видеть': 2, 'рядом': 1, 'муж': 1, 'отправляться': 1, 'на': 10, 'он': 3, 'поиск': 1, 'дом': 2, 'она': 2, 'открывать': 1, 'пожилой': 1, 'мужчина': 1, 'тут': 1, 'начинаться': 1, 'вообще': 3, 'непонятный': 1, 'книга': 18, 'я': 7, 'расстроить': 1, 'полностью': 1, 'такой': 4, 'красивый': 2, 'оформление': 2, 'приятный': 2, 'ощупь': 1, 'обложка': 1, 'качественный': 1, 'бумага': 2, 'отличный': 1, 'шрифт': 1, 'содержани

In [87]:
li_all_pos = set(list(pos_bag.keys()))
li_all_neg = set(list(neg_bag.keys()))

In [88]:
li_only_pos = li_all_pos - (li_all_pos & li_all_neg)
li_only_pos

{'абдуллаев',
 'адекватный',
 'академик',
 'актуальный',
 'александров',
 'алексеев',
 'альбом',
 'америка',
 'анализ',
 'англ',
 'артур',
 'атмосфера',
 'атмосферный',
 'бал',
 'без',
 'безалаберный',
 'безукоризненный',
 'безумно',
 'белый',
 'бизнесмен',
 'билли',
 'благодарить',
 'благодарность',
 'благодаря',
 'блестящий',
 'блин',
 'бог',
 'более',
 'болотный',
 'большой',
 'бонус',
 'босха',
 'боярыня',
 'боярышня',
 'бросаться',
 'брюнетка',
 'будто',
 'буква',
 'бумажный',
 'бы',
 'былинный',
 'бытовой',
 'важно',
 'вариант',
 'вверху',
 'вдохновляться',
 'ведьма',
 'великий',
 'вера',
 'верить',
 'вес',
 'весьма',
 'вечный',
 'вещь',
 'взахлёб',
 'взгляд',
 'взрослый',
 'взять',
 'взяться',
 'вид',
 'виммельбух',
 'виновник',
 'виньетка',
 'власть',
 'влипать',
 'вместе',
 'вместо',
 'вмятина',
 'внизу',
 'внимание',
 'вовсе',
 'возвращать',
 'воздух',
 'возможно',
 'возможность',
 'воин',
 'война',
 'восторг',
 'впервые',
 'впечатление',
 'вполне',
 'всевозможный',
 'всегда'

In [89]:
li_only_neg = li_all_neg - (li_all_pos & li_all_neg)
li_only_neg

{'абьюз',
 'алкоголь',
 'альфа',
 'аста',
 'бабло',
 'балл',
 'бедный',
 'бесплатный',
 'бестселлер',
 'болезнь',
 'больной',
 'большинство',
 'бред',
 'буря',
 'быстрый',
 'взглянуть',
 'взятка',
 'вкусный',
 'волна',
 'выливаться',
 'вынос',
 'вырвать',
 'выслушать',
 'высосать',
 'гельминт',
 'гнести',
 'гнетущий',
 'гопник',
 'горе',
 'городской',
 'господин',
 'графоманя',
 'гэри',
 'дед',
 'действие',
 'деньга',
 'детский',
 'джейкоб',
 'диван',
 'документ',
 'доллар',
 'дом',
 'домашний',
 'драться',
 'ехать',
 'жалко',
 'журнал',
 'забацать',
 'забыть',
 'завтра',
 'закончить',
 'заносить',
 'запоем',
 'зачем',
 'зож',
 'зуб',
 'известный',
 'издали',
 'именитый',
 'иначе',
 'информативность',
 'искусство',
 'испарение',
 'канун',
 'кент',
 'кибардин',
 'кислющий',
 'конец',
 'конфетка',
 'копеечный',
 'кратко',
 'критик',
 'кювета',
 'ломать',
 'лютый',
 'магия',
 'марля',
 'минус',
 'мозг',
 'мороз',
 'муж',
 'название',
 'назвать',
 'называться',
 'напихать',
 'направление',

Выкинем слова, встречающиеся 1-2 раза:

In [90]:
pos_bag_small = {}
for k, v in pos_bag.items():
    if v >= 3:
        pos_bag_small[k] = v
print(pos_bag_small)

{'и': 106, 'очень': 35, 'книга': 72, 'о': 22, 'родитель': 3, 'чудесный': 3, 'разный': 3, 'от': 8, 'нет': 6, 'читать': 11, 'в': 73, 'первый': 5, 'который': 8, 'по': 16, 'то': 6, 'дело': 3, 'история': 7, 'но': 24, 'вот': 3, 'не': 42, 'мочь': 5, 'кстати': 3, 'я': 39, 'решить': 4, 'свой': 5, 'издание': 9, 'любимый': 3, 'хотеть': 3, 'приобрести': 3, 'перевод': 4, 'оформление': 3, 'иллюстрация': 14, 'это': 30, 'так': 12, 'что': 32, 'именно': 4, 'он': 17, 'даже': 6, 'издательство': 4, 'год': 4, 'всё': 22, 'достаточно': 7, 'рочестер': 4, 'с': 35, 'глаз': 4, 'а': 9, 'вообще': 3, 'при': 4, 'ребёнок': 24, 'сам': 7, 'отличный': 8, 'ещё': 6, 'есть': 10, 'удобно': 3, 'страница': 10, 'шрифт': 11, 'быть': 19, 'крупный': 6, 'чёткий': 6, 'качество': 12, 'печать': 4, 'бумага': 14, 'белый': 6, 'плотный': 3, 'довольный': 6, 'рекомендовать': 6, 'к': 16, 'покупка': 3, 'фото': 6, 'просто': 8, 'газета': 4, 'весь': 10, 'мир': 5, 'за': 11, 'германия': 6, 'другой': 7, 'как': 20, 'купить': 5, 'хороший': 22, 'тольк

In [91]:
neg_bag_small = {}
for k, v in neg_bag.items():
    if v >= 3:
        neg_bag_small[k] = v
print(neg_bag_small)

{'от': 3, 'по': 7, 'и': 28, 'в': 20, 'но': 5, 'они': 6, 'чтобы': 5, 'свой': 3, 'история': 4, 'о': 3, 'не': 16, 'с': 5, 'себя': 3, 'на': 10, 'он': 3, 'вообще': 3, 'книга': 18, 'я': 7, 'такой': 4, 'это': 8, 'что': 8, 'читать': 3, 'можно': 3, 'за': 4, 'про': 3, 'если': 5, 'вы': 5, 'техника': 3, 'то': 3, 'этот': 4, 'для': 3, 'знать': 3, 'быть': 3, 'очень': 3, 'тот': 5, 'страница': 4, 'как': 4, 'ещё': 3, 'а': 7, 'попасть': 3, 'из': 4, 'автор': 4, 'всё': 3, 'у': 3, 'скандинавский': 3, 'ходьба': 3}


In [92]:
li_all_pos = set(list(pos_bag_small.keys()))
li_all_neg = set(list(neg_bag_small.keys()))

In [93]:
li_only_pos = li_all_pos - (li_all_pos & li_all_neg)
li_only_pos

{'белый',
 'большой',
 'брать',
 'буква',
 'бумага',
 'вариант',
 'ваш',
 'весь',
 'внимание',
 'вот',
 'время',
 'второй',
 'газета',
 'германия',
 'герой',
 'глаз',
 'год',
 'давать',
 'даже',
 'далёкий',
 'данный',
 'дать',
 'дело',
 'до',
 'добрый',
 'довольный',
 'достаточно',
 'дочь',
 'дронго',
 'другой',
 'единственный',
 'есть',
 'ждать',
 'же',
 'желание',
 'задание',
 'заниматься',
 'издание',
 'издательство',
 'иллюстрация',
 'именно',
 'интерес',
 'интересный',
 'интрига',
 'к',
 'каждый',
 'карта',
 'качество',
 'когда',
 'который',
 'красочный',
 'крупный',
 'кстати',
 'кто',
 'купить',
 'литературный',
 'любимый',
 'маленький',
 'мир',
 'мой',
 'мочь',
 'мы',
 'мягкий',
 'написать',
 'наш',
 'небольшой',
 'немой',
 'несколько',
 'нет',
 'нравиться',
 'обложка',
 'обычно',
 'один',
 'она',
 'остальной',
 'отличный',
 'оформление',
 'оценка',
 'первый',
 'перевод',
 'перед',
 'печать',
 'плотный',
 'подготовка',
 'поездка',
 'покупка',
 'полезный',
 'понравиться',
 'понят

In [94]:
li_only_neg = li_all_neg - (li_all_pos & li_all_neg)
li_only_neg

{'попасть', 'про', 'скандинавский', 'техника', 'ходьба'}

In [95]:
def classify(text):
    words = []
    pos_num = 0
    neg_num = 0
    text = re.sub('<.*>', '', text)
    tokens = nltk.word_tokenize(text)
    for token in tokens:
        if token.isalpha():
            token = token.lower()
            parsed = morph.parse(token)[0]
            words.append(parsed.normal_form)
    for word in words:
        if word in li_only_pos:
            pos_num += 1
        elif word in li_only_neg:
            neg_num += 1
    if pos_num >= neg_num:
        text_class = 'positive'
    else:
        text_class = 'negative'
    return(text_class)

(Так как классификация бинарная, в случае, если слов, встречащихся только в положительных и только в отрицательных отзывах, окажется поровну, будем определять класс как "положительный")

Приготовим тестовые данные.

In [96]:
pos_test = data[data['note'] == 'positive'][61:71]
len(pos_test)

10

In [97]:
neg_test = data[data['note'] == 'negative'][10:]
len(neg_test)

10

In [98]:
neg_test

,review,note
128,"Жвалевского и Пастернак хотела почитать давно,...",negative
130,"Нудный и бессвязный поток сознания врача, реши...",negative
136,"Это лженаука, русофобское произведение и мне о...",negative
144,"По моему мнению начало было хорошим , интересн...",negative
150,Здравствуйте. Я прочла аннотацию книги и захот...,negative
161,К самой книге как к изданию вопросов нет - хор...,negative
166,"Я не понимаю, как такой сборник вообще допусти...",negative
168,Купилась на слово круиз и хвалебные отзывы. Оч...,negative
188,Книга - откровенная дрянь! Не советую.,negative
217,Это ужасно!!! Вот прямо разочаровал автор. Нап...,negative


In [99]:
test = pd.concat([pos_test, neg_test])

In [100]:
test

,review,note
74,"Книга топ , качество печати дно",positive
75,Добрый день) Это просто ШИКАРНОЕ произведение....,positive
77,Любимая серия книг у моего 9-ти летнего сына. ...,positive
79,Старые добрые сказки в формате карманной книги...,positive
81,Мемуары великой княгини Ольги Александровны - ...,positive
82,Это произведение в сто крат лучше Войны и мира...,positive
83,"Приобреталась книга для сына 3,5 лет. И стала ...",positive
85,Очень рекомендую. Такая добрая и светлая книга...,positive
86,Качество печати отличное. Короткие рассказы во...,positive
91,Восхитительная книга! На самом деле до конца т...,positive


In [101]:
output = []
for text in test['review']:
    output.append(classify(text))

In [102]:
output

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive']

In [103]:
from sklearn.metrics import accuracy_score
accuracy_score(list(test['note']), output)

0.5

**Как можно улучшить программу?**

1. Убрать стоп-слова русского языка (из-за маленькой выборки в словах, встречающихся только в негативных отзывых, есть местоимения, например)
2. Классифицировать отзывы с помощью эмбеддингов (word2vec)
3. Взять отзывы на книги одного жанра; отзывы, примерно одинаковые по объёму
4. Сделать не бинарную классификацию, а многоклассовую (оценки на "Лабиринте" -- изначально 'Плохо', 'Лучше не бывает!', 'Очень плохо', 'Средне', 'Отлично', 'Хорошо', 'Хуже не бывает!', 'Нормально', 'Выше среднего', 'Ниже среднего'). 
5. Выделить в негативных и позитивных отзывах ключевые слова с помощью tf-idf, в отзывах для классификации сначала смотреть на ключевые слова, а уже потом на все уникальные (как сейчас)
6. Если слово из тестового отзыва встречается в уникальных словах негативных/позитивных отзывов, добавлять не 1 к "результату" негативного/позитивного отзыва (как сейчас в программе), а добавлять 1 умноженную на коэффициент. Коэффициент тем выше, чем выше частотность слова в отзывах. Так, случайное совпадение будет прибавлять мало, а (условно) слово "чудесный", встречающееся в уникальных словах положительных отзывов много раз, будет прибавлять много. Другими словами, добавить в функцию классификации веса. (Но на наших маленьких данных это, скорее всего, нерелевантно. Случаев, когда pos_num == neg_num, не было.)

Применим улучшение 1, чтобы улучшить программу, а не переписать её заново..

In [106]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [107]:
li_only_pos_clean = []
li_only_neg_clean = []
for word in li_only_pos:
    if word not in russian_stopwords:
        li_only_pos_clean.append(word)
for word in li_only_neg:
    if word not in russian_stopwords:
        li_only_neg_clean.append(word)

In [108]:
print(len(li_only_neg), len(li_only_neg_clean))

5 4


In [109]:
print(len(li_only_pos), len(li_only_pos_clean))

141 121


In [110]:
def classify(text):
    words = []
    pos_num = 0
    neg_num = 0
    text = re.sub('<.*>', '', text)
    tokens = nltk.word_tokenize(text)
    for token in tokens:
        if token.isalpha():
            token = token.lower()
            parsed = morph.parse(token)[0]
            words.append(parsed.normal_form)
    for word in words:
        if word in li_only_pos_clean:
            pos_num += 1
        elif word in li_only_neg_clean:
            neg_num += 1
    if pos_num >= neg_num:
        text_class = 'positive'
    else:
        text_class = 'negative'
    return(text_class)

In [111]:
output = []
for text in test['review']:
    output.append(classify(text))
accuracy_score(list(test['note']), output)

0.5

Улучшение не сработало, но на таких маленьких данных это случайный эффект. (Другими словами, хуже тоже не стало).

**Тогда сделаем tf-idf (улучшение 5).**

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer
stops = stopwords.words("russian")

pos_tfidf = TfidfVectorizer(analyzer="word", stop_words=stops)
neg_tfidf = TfidfVectorizer(analyzer="word", stop_words=stops)

In [113]:
pos_tfidf_texts = []
for row in pos['review'][:60]:
    words = []
    row = re.sub('<.*>', '', row)
    tokens = nltk.word_tokenize(row)
    for token in tokens:
        if token.isalpha():
            token = token.lower()
            parsed = morph.parse(token)[0]
            words.append(parsed.normal_form)
    line = " ".join(words)
    pos_tfidf_texts.append(line)
            
print(pos_tfidf_texts)

['сильно и очень глубоко книга о необходимость личный свобода подросток о токсичный родитель о чудесный родитель и о очень разный одноклассник о педагог от бог и нет читать обязательно в первый очередь читать родитель', 'очередной весьма приличный триллер про доблестный ричер который путешествовать по америка то и дело влипать в разный криминальный история но всегда побеждать вот и тут история не мочь не завершиться поражение злодей среди который кстати фигурировать и злодейский украинец', 'я решить обновить свой старое издание любимый книга и хотеть приобрести в перевод но оформление и иллюстрация это издание я так покорить что я решить приобрести именно он пусть даже и в перевод станевич книга выпустить в издательство качели в год иллюстрация рисовать художник екатерина комраков надо сказать что иллюстрация в это издание отличаться тем что всё персонаж изобразить очень симпатичный и достаточно миловидный или красивый даже миссис рочестер нарисовать привлекательный брюнетка с голубой 

In [114]:
len(pos_tfidf_texts)

60

In [115]:
neg_tfidf_texts = []
for row in neg['review'][:60]:
    words = []
    row = re.sub('<.*>', '', row)
    tokens = nltk.word_tokenize(row)
    for token in tokens:
        if token.isalpha():
            token = token.lower()
            parsed = morph.parse(token)[0]
            words.append(parsed.normal_form)
    line = " ".join(words)
    neg_tfidf_texts.append(line)
            
print(neg_tfidf_texts)

['романтический новелла от эксперт по отношение гэри чепмена марля и джейкоб пожениться в канун рождество но спустя год решить развестись они ехать к юрист чтобы закончить свой история и подписать необходимый документ о развод попадать в снежный буря и джейкоб не справляться с управление они заносить они уходить в кювета марля приходить в себя не видеть рядом муж отправляться на он поиск видеть дом она открывать пожилой мужчина и тут начинаться вообще непонятный книга я расстроить полностью такой красивый оформление приятный на ощупь обложка качественный бумага и отличный шрифт но содержание это вынос мозг вообще не понятно что я читать радовать что прочитать книга можно за час и забыть про она общий балл от я и это только за оформление книга', 'честно говорить разочаровать данный книга скорее можно назвать справочник если вы разбираться в искусство и вы нужный шпаргалка по техника направление произведение то этот книга для вы если вы ничего не знать о искусство то информация в книга б

In [116]:
len(neg_tfidf_texts)

10

In [117]:
pos_reviews_tfidf = pos_tfidf.fit_transform(pos_tfidf_texts)
neg_reviews_tfidf = neg_tfidf.fit_transform(neg_tfidf_texts)

In [118]:
pos_reviews_tfidf

<60x884 sparse matrix of type '<class 'numpy.float64'>'
	with 1509 stored elements in Compressed Sparse Row format>

In [119]:
neg_reviews_tfidf

<10x317 sparse matrix of type '<class 'numpy.float64'>'
	with 362 stored elements in Compressed Sparse Row format>

In [120]:
pos_tfidf.get_feature_names()[:10]

C:\Users\nasty\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['абдуллаев',
 'автор',
 'адекватный',
 'академик',
 'актуальный',
 'александров',
 'алексеев',
 'альбом',
 'америка',
 'анализ']

In [121]:
neg_tfidf.get_feature_names()[:10]

['абьюз',
 'автор',
 'алкоголь',
 'альфа',
 'аста',
 'бабло',
 'балл',
 'бедный',
 'бесплатный',
 'бестселлер']

In [122]:
def get_key_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

In [123]:
pos_reviews_tfidf

<60x884 sparse matrix of type '<class 'numpy.float64'>'
	with 1509 stored elements in Compressed Sparse Row format>

In [124]:
feature_names = np.array(pos_tfidf.get_feature_names())
pos_words = get_key_words(pos_reviews_tfidf, feature_names, 40)
print(pos_words)

['босха' 'советовать' 'окончание' 'прочтение' 'советовать' 'родитель'
 'глава' 'прочтение' 'самый' 'карта' 'второй' 'отличный' 'обложка'
 'первый' 'наглядный' 'ознакомление' 'выкладывать' 'фотография' 'обложка'
 'дронго' 'слово' 'который' 'показатель' 'германия' 'рассматривать'
 'прекрасно' 'справляться' 'понравиться' 'хороший' 'это' 'инструмент'
 'второй' 'читать' 'сын' 'умный' 'учебник' 'учиться' 'пятёрка'
 'предсказуемый' 'приятельница']


In [125]:
feature_names = np.array(neg_tfidf.get_feature_names())
neg_vector = neg_reviews_tfidf
neg_words = get_key_words(neg_vector, feature_names, 40)
print(neg_words)

['опечатка' 'слабый' 'перевод' 'информативность' 'неочень' 'качество'
 'ходьба' 'скандинавский' 'страница' 'совершенно' 'показаться' 'скучный'
 'сюжет' 'главный' 'герой' 'сопереживать' 'ребёнок' 'друг' 'искусство'
 'разбираться' 'хотеться' 'книга' 'весь' 'очень' 'всё' 'история'
 'представлять' 'повествование' 'шизофреник' 'русофоб' 'научный' 'труд'
 'оценка' 'ставить' 'рецензия' 'писать' 'видимо' 'понасенков'
 'публицистический' 'господин']


(Слова странные за счёт маленькой выборки, но ладно)

In [126]:
def classify(text):
    words = []
    pos_num = 0
    neg_num = 0
    text = re.sub('<.*>', '', text)
    tokens = nltk.word_tokenize(text)
    for token in tokens:
        if token.isalpha():
            token = token.lower()
            parsed = morph.parse(token)[0]
            words.append(parsed.normal_form)
    
    for word in words:
        if word in list(pos_words):
            pos_num += 1
        elif word in (neg_words):
            neg_num += 1
        if word in li_only_pos:
            pos_num += 1
        elif word in li_only_neg:
            neg_num += 1
    if pos_num >= neg_num:
        text_class = 'positive'
    else:
        text_class = 'negative'
    return(text_class)

In [127]:
list(pos_words)

['босха',
 'советовать',
 'окончание',
 'прочтение',
 'советовать',
 'родитель',
 'глава',
 'прочтение',
 'самый',
 'карта',
 'второй',
 'отличный',
 'обложка',
 'первый',
 'наглядный',
 'ознакомление',
 'выкладывать',
 'фотография',
 'обложка',
 'дронго',
 'слово',
 'который',
 'показатель',
 'германия',
 'рассматривать',
 'прекрасно',
 'справляться',
 'понравиться',
 'хороший',
 'это',
 'инструмент',
 'второй',
 'читать',
 'сын',
 'умный',
 'учебник',
 'учиться',
 'пятёрка',
 'предсказуемый',
 'приятельница']

In [128]:
output = []
for text in test['review']:
    output.append(classify(text))

In [129]:
output

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive']

In [130]:
len(output)

20

In [131]:
from sklearn.metrics import accuracy_score
accuracy_score(list(test['note']), output)

0.5

Ура, качество немного улучшилось (было 0.7)! Интересно, что отдельно опора на самые частотные в tf-idf (по всем положительным/отрицательным отзывам) работает хуже, а вместе с идеей из условия дз -- лучше. Неравные коэффциенты (не 1) ухудшают качество (я попробовала). От количества ключевых слов качество также зависит, я пробовала 10, 20, 30, 40 и 50. 40 -- оптимальное, хотя при 30 и 50 accuracy = 0.8, на наших маленьких данных эти различия можно считать статистически незначимыми.
(Функция немного странная, потому что я старалась не совсем её переписывать, а улучшить её учётом ключевых слов всех положительных/отрицательных отзывов при классификации.)

**Домашнее задание 2:**

Мне пришлось перезапустить код, и в новой версии набралось меньше негативных отзывов, поэтому качество "изначальной" программы упало с 0,7 до 0,5. (Времени снова ждать, чтобы набралось много отрицательных отзывов, у меня, к сожалению, в этот раз нет.) Что было 0,7, можно посмотреть в репозитории первой дз. Но всё равно попробуем применить чанкер теперь.

In [133]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [134]:
def chunker(seq, size):
    output = []
    for element in range(0, len(seq)-size+1):
        output.append(seq[element:element + size])
    return output

In [135]:
def ngrams(text):
    res = []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for group in chunker(doc.tokens, 2):
        if group[0].pos == 'ADV' and group[1].pos == 'ADJ':
            res.append(' '.join([group[0].text, group[1].text]))
        if group[0].pos == 'ADJ' and group[1].pos == 'NOUN':
            res.append(' '.join([group[0].text, group[1].text]))
    for group in chunker(doc.tokens, 3):
        if group[0].pos == 'VERB' and group[1].pos == 'ADP' and group[2].pos == 'NOUN':
            res.append(' '.join([group[0].text, group[1].text, group[2].text]))
    return res

In [139]:
pos_bag = {}
for row in pos['review']:
    row = re.sub('<.*>', '', row)
    row_ngrams = ngrams(row)
    tokens = nltk.word_tokenize(row)
    for token in tokens:
        if token.isalpha():
            token = token.lower()
            parsed = morph.parse(token)[0]
            if parsed.normal_form in pos_bag:
                pos_bag[parsed.normal_form] += 1
            else:
                pos_bag[parsed.normal_form] = 1
    for ngram in row_ngrams:
        if ngram in pos_bag:
            pos_bag[ngram] += 1
        else:
            pos_bag[ngram] = 1
            
print(pos_bag)

{'сильно': 1, 'и': 106, 'очень': 35, 'глубоко': 1, 'книга': 72, 'о': 22, 'необходимость': 1, 'личный': 1, 'свобода': 1, 'подросток': 1, 'токсичный': 1, 'родитель': 3, 'чудесный': 3, 'разный': 3, 'одноклассник': 1, 'педагог': 1, 'от': 8, 'бог': 1, 'нет': 6, 'читать': 11, 'обязательно': 1, 'в': 73, 'первый': 5, 'очередь': 1, 'личной свободы': 1, 'токсичных родителях': 1, 'чудесных родителях': 1, 'очень разных': 1, 'разных одноклассниках': 1, 'первую очередь': 1, 'очередной': 1, 'весьма': 1, 'приличный': 1, 'триллер': 1, 'про': 1, 'доблестный': 1, 'ричер': 1, 'который': 8, 'путешествовать': 1, 'по': 16, 'америка': 1, 'то': 6, 'дело': 3, 'влипать': 1, 'криминальный': 1, 'история': 7, 'но': 24, 'всегда': 2, 'побеждать': 1, 'вот': 3, 'тут': 1, 'не': 42, 'мочь': 5, 'завершиться': 1, 'поражение': 1, 'злодей': 1, 'среди': 1, 'кстати': 3, 'фигурировать': 1, 'злодейский': 1, 'украинец': 1, 'весьма приличный': 1, 'приличный триллер': 1, 'криминальные истории': 1, 'злодейский украинец': 1, 'я': 39,

In [140]:
neg_bag = {}
for row in neg['review']:
    row = re.sub('<.*>', '', row)
    row_ngrams = ngrams(row)
    tokens = nltk.word_tokenize(row)
    for token in tokens:
        if token.isalpha():
            token = token.lower()
            parsed = morph.parse(token)[0]
            if parsed.normal_form in neg_bag:
                neg_bag[parsed.normal_form] += 1
            else:
                neg_bag[parsed.normal_form] = 1
    for ngram in row_ngrams:
        if ngram in neg_bag:
            neg_bag[ngram] += 1
        else:
            neg_bag[ngram] = 1
            
print(neg_bag)

{'романтический': 1, 'новелла': 1, 'от': 3, 'эксперт': 1, 'по': 7, 'отношение': 1, 'гэри': 1, 'чепмена': 1, 'марля': 2, 'и': 28, 'джейкоб': 2, 'пожениться': 1, 'в': 20, 'канун': 1, 'рождество': 1, 'но': 5, 'спустя': 1, 'год': 1, 'решить': 1, 'развестись': 1, 'они': 6, 'ехать': 1, 'к': 2, 'юрист': 1, 'чтобы': 5, 'закончить': 1, 'свой': 3, 'история': 4, 'подписать': 1, 'необходимый': 1, 'документ': 1, 'о': 3, 'развод': 1, 'попадать': 1, 'снежный': 1, 'буря': 1, 'не': 16, 'справляться': 1, 'с': 5, 'управление': 1, 'заносить': 1, 'уходить': 1, 'кювета': 1, 'приходить': 1, 'себя': 3, 'видеть': 2, 'рядом': 1, 'муж': 1, 'отправляться': 1, 'на': 10, 'он': 3, 'поиск': 1, 'дом': 2, 'она': 2, 'открывать': 1, 'пожилой': 1, 'мужчина': 1, 'тут': 1, 'начинаться': 1, 'вообще': 3, 'непонятный': 1, 'книга': 18, 'я': 7, 'расстроить': 1, 'полностью': 1, 'такой': 4, 'красивый': 2, 'оформление': 2, 'приятный': 2, 'ощупь': 1, 'обложка': 1, 'качественный': 1, 'бумага': 2, 'отличный': 1, 'шрифт': 1, 'содержани

In [141]:
li_all_pos = set(list(pos_bag.keys()))
li_all_neg = set(list(neg_bag.keys()))

In [142]:
li_only_pos = li_all_pos - (li_all_pos & li_all_neg)
li_only_pos

{'попробовать',
 'впервые',
 'прекрасный',
 'получить',
 'счёт',
 'привлекательный маг',
 'прошить',
 'перенести',
 'еще маленький',
 'чингиз',
 'добывать',
 'потрясать',
 'убийство',
 'слишком',
 'Русский язык',
 'отзыв',
 'подозревать',
 'разных одноклассниках',
 'очень тонкая',
 'перекроить',
 'без',
 'Единственным недостатком',
 'над',
 'влипать',
 'очень важно',
 'найти',
 'срок',
 'педагог',
 'инструмент',
 'вмятина',
 'прикупить в переводе',
 'паранойя',
 'любить',
 'крошечных фей',
 'чудесных родителях',
 'однотипный',
 'увесистый',
 'полезный',
 'спасибо',
 'первую очередь',
 'очень плотная',
 'любитель',
 'проявлять',
 'преимущественно маленькие',
 'фотография',
 'других исследований',
 'украинец',
 'Очень хороший',
 'благодарность',
 'осмыслить',
 'лёгкий',
 'очень добрые',
 'очень симпатичными',
 'возможно',
 'исполнять',
 'отлично',
 'марь',
 'пролог',
 'недавно',
 'интересная составляющая',
 'интересным сюжетом',
 'развитие',
 'изменялось в угоду',
 'повседневных делах',


In [143]:
li_only_neg = li_all_neg - (li_all_pos & li_all_neg)
li_only_neg

{'Большего бреда',
 'Городская проза',
 'Данную книгу',
 'Неочень книга',
 'Неплохой сборник',
 'Общий балл',
 'Романтическая новелла',
 'абьюз',
 'алкоголь',
 'альфа',
 'аста',
 'бабло',
 'балл',
 'бедный',
 'бесплатного отрывка',
 'бесплатный',
 'бестселлер',
 'болезнь',
 'больной',
 'большинство',
 'бред',
 'буря',
 'быстрый',
 'взглянула на название',
 'взглянуть',
 'взятка',
 'вкусный',
 'волна',
 'вообще непонятное',
 'выливается на бумагу',
 'выливаться',
 'вынос',
 'вырвать',
 'выслушать',
 'высосанный из пальца',
 'высосать',
 'гельминт',
 'главному герою',
 'гнести',
 'гнетущий',
 'гопник',
 'горе',
 'городской',
 'господин',
 'графоманя',
 'гэри',
 'данный опус',
 'дед',
 'действие',
 'деньга',
 'детский',
 'детской книжки',
 'джейкоб',
 'диван',
 'документ',
 'доллар',
 'дом',
 'домашнее насилие',
 'домашний',
 'дочитает до конца',
 'драться',
 'едут к юристу',
 'ехать',
 'жалко',
 'живущими в фурункулах',
 'журнал',
 'забацать',
 'забацать бестселлер',
 'забыть',
 'завтра'

In [145]:
output = []
for text in test['review']:
    output.append(classify(text))

In [147]:
output

['negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive']

In [148]:
from sklearn.metrics import accuracy_score
accuracy_score(list(test['note']), output)

0.45

Это из-за того, что негативных отзывов почти не набралось((